In [19]:
import os
import logging
from larksuiteoapi import Config, DOMAIN_FEISHU, DefaultLogger, MemoryStore, LEVEL_ERROR, ACCESS_TOKEN_TYPE_TENANT
from larksuiteoapi.api import Request

In [20]:
os.environ["APP_ID"] = "cli_a39db339cdfad00d"
os.environ["APP_SECRET"] = "QRiRvybIxT7XFbQ5HHD6Ocq64dVlP10p"

In [134]:
Config.new_internal_app_settings_from_env??

In [21]:
app_settings = Config.new_internal_app_settings_from_env()
conf = Config(DOMAIN_FEISHU, app_settings, logger=DefaultLogger(), log_level=LEVEL_ERROR, store=MemoryStore())

In [131]:
type(conf)

larksuiteoapi.config.Config

## 获取表格元数据

In [25]:
http_Method = "GET"
spreadsheetToken = "shtcndrhB04W9FW3XKbqa8RfOIV"
http_path = f"https://open.feishu.cn/open-apis/sheets/v2/spreadsheets/{spreadsheetToken}/metainfo"
access_token_type = ACCESS_TOKEN_TYPE_TENANT
request_body = None

In [26]:
req = Request(http_path, http_Method, access_token_type, request_body, request_opts=None)

In [41]:
resp = req.do(conf)

print('request id = %s' % resp.get_request_id())
print(resp.code)

request id = 2022090814121501015021123006021335
0


In [42]:
if resp.code == 0:
    print(resp.data["sheets"])
else:
    print(resp.msg)
    print(resp.error)

[{'columnCount': 20, 'frozenColCount': 0, 'frozenRowCount': 0, 'index': 0, 'rowCount': 194, 'sheetId': '558383', 'title': 'Sheet1'}, {'columnCount': 20, 'frozenColCount': 0, 'frozenRowCount': 0, 'index': 1, 'rowCount': 200, 'sheetId': 'MdX08r', 'title': 'Sheet2'}]


In [43]:
resp.data["sheets"]

[{'columnCount': 20,
  'frozenColCount': 0,
  'frozenRowCount': 0,
  'index': 0,
  'rowCount': 194,
  'sheetId': '558383',
  'title': 'Sheet1'},
 {'columnCount': 20,
  'frozenColCount': 0,
  'frozenRowCount': 0,
  'index': 1,
  'rowCount': 200,
  'sheetId': 'MdX08r',
  'title': 'Sheet2'}]

In [44]:
sheetId = resp.data["sheets"][0]["sheetId"]

In [45]:
sheetId

'558383'

## 读取单个范围数据

In [105]:
http_Method = "GET"
range = f"{sheetId}!A:I"
http_path = f"https://open.feishu.cn/open-apis/sheets/v2/spreadsheets/{spreadsheetToken}/values/{range}?valueRenderOption=FormattedValue"

In [106]:
req = Request(http_path, http_Method, access_token_type, request_body, request_opts=None)

resp = req.do(conf)

print('request id = %s' % resp.get_request_id())
print(resp.code)

if resp.code == 0:
    print(resp.data)
else:
    print(resp.msg)
    print(resp.error)

request id = 202209081449380102080370960D0D937B
0
{'revision': 41, 'spreadsheetToken': 'shtcndrhB04W9FW3XKbqa8RfOIV', 'valueRange': {'majorDimension': 'ROWS', 'range': '558383!A1:I5', 'revision': 41, 'values': [['Trial Index', 'Trial Type', 'T (C)', 'P (MPa)', 'GR (mL/min)', 'LR (mL/min)', 'C (wt%)', 'THFDM Productivity (mol/h)', 'E-Factor'], [0, 'INIT', 130, 5, 40, 0.1, 5, 0.001326406, 0.430519], [0, 'INIT', 120, 3, 80, 2.0333333335, 5, 0.021001056, 0.795983], [0, 'INIT', 130, 3, 80, 0.422222222, 10, 0.011446429, 0.362433], [0, 'INIT', 120, 3, 60, 1.066666667, 2.5, 0.007536612, 0.285219]]}}


In [114]:
resp.data['valueRange']['values']

[['Trial Index',
  'Trial Type',
  'T (C)',
  'P (MPa)',
  'GR (mL/min)',
  'LR (mL/min)',
  'C (wt%)',
  'THFDM Productivity (mol/h)',
  'E-Factor'],
 [0, 'INIT', 130, 5, 40, 0.1, 5, 0.001326406, 0.430519],
 [0, 'INIT', 120, 3, 80, 2.0333333335, 5, 0.021001056, 0.795983],
 [0, 'INIT', 130, 3, 80, 0.422222222, 10, 0.011446429, 0.362433],
 [0, 'INIT', 120, 3, 60, 1.066666667, 2.5, 0.007536612, 0.285219]]

In [120]:
resp.data['valueRange']['values'][1:]

[[0, 'INIT', 130, 5, 40, 0.1, 5, 0.001326406, 0.430519],
 [0, 'INIT', 120, 3, 80, 2.0333333335, 5, 0.021001056, 0.795983],
 [0, 'INIT', 130, 3, 80, 0.422222222, 10, 0.011446429, 0.362433],
 [0, 'INIT', 120, 3, 60, 1.066666667, 2.5, 0.007536612, 0.285219]]

In [129]:
import numpy as np
import pandas as pd

numpy_array = np.array(resp.data['valueRange']['values'][1:])
numpy_columns = resp.data['valueRange']['values'][0]
df = pd.DataFrame(numpy_array, columns=numpy_columns)

In [130]:
df

,Trial Index,Trial Type,T (C),P (MPa),GR (mL/min),LR (mL/min),C (wt%),THFDM Productivity (mol/h),E-Factor
0,0,INIT,130,5,40,0.1,5,0.001326406,0.430519
1,0,INIT,120,3,80,2.0333333335,5,0.021001056,0.795983
2,0,INIT,130,3,80,0.422222222,10,0.011446429,0.362433
3,0,INIT,120,3,60,1.066666667,2.5,0.007536612,0.285219


## 向单个范围写入数据

In [96]:
http_Method = "PUT"
http_path = f"https://open.feishu.cn/open-apis/sheets/v2/spreadsheets/{spreadsheetToken}/values"

In [102]:
request_body = {
    "valueRange":{
        "range": "558383!A2:I",
        "values": [
            [0, "INIT", 130, 5, 40, 0.1, 5, 0.001326406, 0.430519],
            [0, "INIT", 120, 3, 80, 2.0333333335, 5, 0.021001056, 0.795983],
            [0, "INIT", 130, 3, 80, 0.422222222, 10, 0.011446429, 0.362433],
            [0, "INIT", 120, 3, 60, 1.066666667, 2.5, 0.007536612, 0.285219]
        ]
    }
}

In [103]:
req = Request(http_path, http_Method, access_token_type, request_body, request_opts=None)

resp = req.do(conf)

print('request id = %s' % resp.get_request_id())
print(resp.code)

request id = 20220908144856010208165096110A5EDE
0


In [104]:
if resp.code == 0:
    print(resp.data)
else:
    print(resp.msg)
    print(resp.error)

{'revision': 41, 'spreadsheetToken': 'shtcndrhB04W9FW3XKbqa8RfOIV', 'updatedCells': 36, 'updatedColumns': 9, 'updatedRange': '558383!A2:I5', 'updatedRows': 4}
